In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
assignment_path = "/content/drive/My Drive/Urban/"
sys.path.append(assignment_path)

In [ ]:
!pip install python-dateutil
!pip install pysal --quiet
from pysal.explore import esda
from pysal.lib import weights
from libpysal.weights.raster import da2W

     |████████████████████████████████| 2.4 MB 7.1 MB/s 
     |████████████████████████████████| 95 kB 3.8 MB/s 
     |████████████████████████████████| 60 kB 7.8 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 164 kB 67.6 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 41 kB 602 kB/s 
     |████████████████████████████████| 209 kB 44.3 MB/s 
     |████████████████████████████████| 5.7 MB 52.9 MB/s 
     |████████████████████████████████| 81 kB 8.6 MB/s 
     |████████████████████████████████| 138 kB 57.1 MB/s 
     |████████████████████████████████| 226 kB 57.6 MB/s 
     |████████████████████████████████| 269 kB 51.5 MB/s 
     |████████████████████████████████| 66.5 MB 1.4 MB/s 
     |████████████████████████████████| 2.1 MB 43.4 MB/s 
     |████████████████████████████████| 183 kB 52.5 MB/s 
     |████████████████████████████████| 117 kB 53.1 MB/s 
     |█████████████████████

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
/usr/local/lib/python3.7/dist-packages/spaghetti/network.py:36: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geo

In [ ]:
!pip install git+git://github.com/geopandas/geopandas.git

  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-tu5iawpn
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-tu5iawpn
  Created wheel for geopandas: filename=geopandas-0.10.2+15.g554d2ee-py2.py3-none-any.whl size=1031778 sha256=acd270f5f6fc5d756eda5512bfe48e02636fad16d627824bd182630db446b9c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-y8mjkr6t/wheels/cf/3e/0b/6475054094c2b1ea054158ac1fdcf749fb92f5b512377e4cf8
Successfully built geopandas
  Attempting uninstall: geopandas
    Found existing installation: geopandas 0.10.2
    Uninstalling geopandas-0.10.2:
      Successfully uninstalled geopandas-0.10.2


In [ ]:
import os, sys
import pandas as pd
import geopandas
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import datetime as date
import warnings
from datetime import datetime
from dateutil import parser
from datetime import timedelta
import pytz

sys.path.insert(0,'../')
%load_ext autoreload
%autoreload 2

warnings.filterwarnings('ignore')

In [1]:
dt = pd.read_csv('/content/drive/My Drive/Urban/Data/311_City_Service_Requests_in_2020.csv', parse_dates=['ADDDATE', 'RESOLUTIONDATE', 'SERVICEDUEDATE', 'SERVICEORDERDATE', 'INSPECTIONDATE'])

cutoffDate = pd.to_datetime(date.date(2020,4,1))
tz = pytz.timezone("UTC")
cutoffDate = tz.localize(cutoffDate)
dt=dt[dt['ADDDATE']<cutoffDate]
dt=dt[dt['ORGANIZATIONACRONYM']=='DPW']
dt['RESOLUTIONDATE']=np.where(dt['RESOLUTIONDATE']>cutoffDate, None, dt['RESOLUTIONDATE'])

dt['DURATION']= np.where(dt['RESOLUTIONDATE'].notna(),np.log(((dt['RESOLUTIONDATE'] - dt['ADDDATE']).dt.total_seconds())/24/60/60),np.log(((cutoffDate - dt['ADDDATE']).dt.total_seconds())/24/60/60))
dt['DURATION']= np.where((dt['DURATION']<0.01), 0.01, dt['DURATION'])

dt.tail(5)

NameError: ignored

In [ ]:
openRequests=dt[dt['RESOLUTIONDATE'].isna()]
print(openRequests)

                X          Y  OBJECTID  ... WARD DETAILS  DURATION
197    -76.986051  38.869008      1414  ...  8.0     NaN  4.503499
228    -77.088417  38.939089      1445  ...  3.0     NaN  4.502899
235    -76.986379  38.869019      1452  ...  8.0     NaN  4.502775
441    -76.986051  38.869008      1658  ...  8.0     NaN  4.503892
904    -76.935021  38.873747      2121  ...  7.0     NaN  4.494904
...           ...        ...       ...  ...  ...     ...       ...
115330 -77.069574  38.950946    176450  ...  3.0     NaN  3.725952
138301 -77.029398  38.932055    223341  ...  1.0     NaN  4.481749
138306 -77.071881  38.916132    223346  ...  2.0     NaN  4.373665
138402 -76.995926  38.879834    223442  ...  6.0     NaN  2.485169
138482 -77.065280  38.965115    223522  ...  4.0     NaN  4.346743

[3257 rows x 31 columns]


In [ ]:
cleanup = {'11', 'GRAFF', 'NEIGHCLEANUP', 'RATREPLACECONTRS', 'S0011', 'S0196', 'S0391', 'SO392'}
collection={'C62313','C62313m','S0031','S0441','S0476','S0321','S0323'}
delRepRem={'CONTREMO','RECCONRE','RECSCHPR','S0216','S0316','S0322','S0422','S0423','S0451','TRACO001'}
snow={'S0383','SIDEWLKEXP','SNOWREMCOMP','SRC01','SRC02'}
vehTic={'BICYCLE','S0000','S0003','S0336','S0261','RPP','EMNPV','S0471','S0181','S0185','S0346'}

openCleanup = openRequests[openRequests['SERVICECODE'].isin(cleanup)]
openCollection = openRequests[openRequests['SERVICECODE'].isin(collection)]
openDelRepRem = openRequests[openRequests['SERVICECODE'].isin(delRepRem)]
openSnow = openRequests[openRequests['SERVICECODE'].isin(snow)]
openVehTic = openRequests[openRequests['SERVICECODE'].isin(vehTic)]
print(openVehTic)

               X          Y  OBJECTID  ... WARD DETAILS  DURATION
197   -76.986051  38.869008      1414  ...  8.0     NaN  4.503499
235   -76.986379  38.869019      1452  ...  8.0     NaN  4.502775
441   -76.986051  38.869008      1658  ...  8.0     NaN  4.503892
997   -77.058225  38.946505      2214  ...  3.0     NaN  4.495857
1399  -76.976750  38.844453      2616  ...  8.0     NaN  4.494195
...          ...        ...       ...  ...  ...     ...       ...
82732 -76.940054  38.894430    116604  ...  7.0     NaN  0.000000
82833 -77.025969  38.962710    116705  ...  4.0     NaN  3.390962
84710 -76.959967  38.856289    119478  ...  7.0     NaN  2.036077
91501 -76.997234  38.943449    129293  ...  5.0     NaN  4.447057
97364 -76.984836  38.919347    138564  ...  5.0     NaN  3.182631

[825 rows x 31 columns]


In [ ]:
geoCleanup=openCleanup[['LATITUDE', 'LONGITUDE']]
openCleanupCluster=OPTICS(max_eps=0.01, min_samples=3).fit(geoCleanup)
#openCleanupCluster=DBSCAN(eps=0.008, min_samples=3).fit(geoCleanup)

geoCollection=openCollection[['LATITUDE', 'LONGITUDE']]
openCollectionCluster=OPTICS(max_eps=0.01, min_samples=3).fit(geoCollection)
#openCollectionCluster=DBSCAN(eps=0.008, min_samples=3).fit(geoCollection)

geoDelRepRem=openDelRepRem[['LATITUDE', 'LONGITUDE']]
openDelRepRemCluster=OPTICS(max_eps=0.01, min_samples=3).fit(geoDelRepRem)
#openDelRepRemCluster=DBSCAN(eps=0.008, min_samples=3).fit(geoDelRepRem)

geoSnow=openSnow[['LATITUDE', 'LONGITUDE']]
openSnowCluster=OPTICS(max_eps=0.01, min_samples=3).fit(geoSnow)
#openSnowCluster=DBSCAN(eps=0.008, min_samples=3).fit(geoSnow)

geoVehTic=openVehTic[['LATITUDE', 'LONGITUDE']]
openVehTicCluster=OPTICS(max_eps=0.01, min_samples=3).fit(geoVehTic)
#openVehTicCluster=DBSCAN(eps=0.008, min_samples=3).fit(geoVehTic)

In [ ]:
cluster_labels = openCleanupCluster.labels_
num_clusters = len(set(cluster_labels))
#print(openCleanupCluster.core_distances_)

labels = openCleanupCluster.labels_

numclusters = len(np.unique(labels) )
numnoise = np.sum(np.array(labels) == -1, axis=0)

print('Cleanup Clusters: %d' % numclusters)
print('Cleanup Noise: %d' % numnoise)

cluster_labels = openCollectionCluster.labels_
num_clusters = len(set(cluster_labels))
#print(openCleanupCluster.core_distances_)

labels = openCollectionCluster.labels_

numclusters = len(np.unique(labels) )
numnoise = np.sum(np.array(labels) == -1, axis=0)

print('Collection Clusters: %d' % numclusters)
print('Collection Noise: %d' % numnoise)

cluster_labels = openDelRepRemCluster.labels_
num_clusters = len(set(cluster_labels))
#print(openCleanupCluster.core_distances_)

labels = openDelRepRemCluster.labels_

numclusters = len(np.unique(labels) )
numnoise = np.sum(np.array(labels) == -1, axis=0)

print('DelRepRem Clusters: %d' % numclusters)
print('DelRepRem Noise: %d' % numnoise)

cluster_labels = openSnowCluster.labels_
num_clusters = len(set(cluster_labels))
#print(openCleanupCluster.core_distances_)

labels = openSnowCluster.labels_

numclusters = len(np.unique(labels) )
numnoise = np.sum(np.array(labels) == -1, axis=0)

print('Snow Clusters: %d' % numclusters)
print('Snow Noise: %d' % numnoise)

cluster_labels = openVehTicCluster.labels_
num_clusters = len(set(cluster_labels))
#print(openCleanupCluster.core_distances_)

labels = openVehTicCluster.labels_

numclusters = len(np.unique(labels) )
numnoise = np.sum(np.array(labels) == -1, axis=0)

print('VehTic Clusters: %d' % numclusters)
print('VehTic Noise: %d' % numnoise)

Cleanup Clusters: 62
Cleanup Noise: 141
Collection Clusters: 238
Collection Noise: 443
DelRepRem Clusters: 61
DelRepRem Noise: 73
Snow Clusters: 19
Snow Noise: 32
VehTic Clusters: 139
VehTic Noise: 174


In [ ]:
closedRequests=dt[dt['RESOLUTIONDATE'].notna()]

closedCleanup = closedRequests[closedRequests['SERVICECODE'].isin(cleanup)]
closedCollection = closedRequests[closedRequests['SERVICECODE'].isin(collection)]
closedDelRepRem = closedRequests[closedRequests['SERVICECODE'].isin(delRepRem)]
closedSnow = closedRequests[closedRequests['SERVICECODE'].isin(snow)]
closedVehTic = closedRequests[closedRequests['SERVICECODE'].isin(vehTic)]

geogroupClosed=closedCleanup[['LATITUDE', 'LONGITUDE']]
closedCleanupCluster=openCleanupCluster.fit_predict(geogroupClosed)

geogroupClosed=closedCollection[['LATITUDE', 'LONGITUDE']]
closedCollectionCluster=openCollectionCluster.fit_predict(geogroupClosed)

geogroupClosed=closedDelRepRem[['LATITUDE', 'LONGITUDE']]
closedDelRepRemCluster=openDelRepRemCluster.fit_predict(geogroupClosed)

geogroupClosed=closedSnow[['LATITUDE', 'LONGITUDE']]
closedSnowCluster=openSnowCluster.fit_predict(geogroupClosed)

geogroupClosed=closedVehTic[['LATITUDE', 'LONGITUDE']]
closedoVehTicCluster=openVehTicCluster.fit_predict(geogroupClosed)

In [ ]:
closedCleanup['cluster']=closedCleanupCluster
closedCollection['cluster']=closedCollectionCluster
closedDelRepRem['cluster']=closedDelRepRemCluster
closedSnow['cluster']=closedSnowCluster
closedVehTic['cluster']=closedoVehTicCluster
closedCleanup.tail(5)

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,SERVICEDUEDATE,SERVICEORDERDATE,INSPECTIONFLAG,INSPECTIONDATE,INSPECTORNAME,SERVICEORDERSTATUS,STATUS_CODE,SERVICEREQUESTID,PRIORITY,STREETADDRESS,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS,DURATION,cluster
81471,-77.027840,38.956374,115039,11,Dead Animal Collection,SWMA- Solid Waste Management Admistration,DPW,1,2020-03-30 15:24:27+00:00,2020-03-31 11:17:04+00:00,2020-04-01 15:24:27+00:00,2020-03-30 15:24:27+00:00,0.0,2020-03-31 11:17:03+00:00,NaN,Closed,NaN,20-00113551,Standard,5501 GEORGIA AVENUE NW,397587.140000,143162.820000,38.956367,-77.027838,WASHINGTON,DC,20011.0,276109.0,4.0,NaN,0.0,333
81502,-77.085207,38.912000,115070,11,Dead Animal Collection,SWMA- Solid Waste Management Admistration,DPW,1,2020-03-30 16:17:50+00:00,2020-03-31 11:14:04+00:00,2020-04-01 16:17:50+00:00,2020-03-30 16:17:50+00:00,0.0,2020-03-31 11:14:03+00:00,NaN,Closed,NaN,20-00113654,Standard,1660 FOXHALL ROAD NW,392610.290000,138239.960000,38.911992,-77.085204,WASHINGTON,DC,20007.0,271443.0,3.0,NaN,0.0,586
81524,-77.080837,38.906541,115092,11,Dead Animal Collection,SWMA- Solid Waste Management Admistration,DPW,1,2020-03-30 16:29:41+00:00,2020-03-31 11:14:08+00:00,2020-04-01 16:29:40+00:00,2020-03-30 16:29:41+00:00,0.0,2020-03-31 11:14:07+00:00,NaN,Closed,NaN,20-00113681,Standard,MACARTHUR BOULEVARD NW AND FOXHALL ROAD NW,392988.761047,137633.673992,38.906533,-77.080834,WASHINGTON,DC,20007.0,906196.0,3.0,NaN,0.0,-1
81676,-77.085050,38.912435,115244,11,Dead Animal Collection,SWMA- Solid Waste Management Admistration,DPW,1,2020-03-30 17:37:54+00:00,2020-03-31 11:13:10+00:00,2020-04-01 17:37:54+00:00,2020-03-30 17:37:54+00:00,0.0,2020-03-31 11:13:09+00:00,NaN,Closed,NaN,20-00113786,Standard,FOXHALL ROAD NW AND RESERVOIR ROAD NW,392623.949751,138288.208328,38.912427,-77.085047,WASHINGTON,DC,20007.0,904548.0,3.0,NaN,0.0,586
82076,-77.037842,38.918545,115852,11,Dead Animal Collection,SWMA- Solid Waste Management Admistration,DPW,1,2020-03-31 01:59:25+00:00,2020-03-31 11:08:04+00:00,2020-04-02 01:59:25+00:00,2020-03-31 01:59:25+00:00,0.0,2020-03-31 11:08:03+00:00,NaN,Closed,NaN,20-00114048,Standard,1638 FLORIDA AVENUE NW,396718.520000,138963.810000,38.918538,-77.037839,WASHINGTON,DC,20009.0,241825.0,1.0,NaN,0.0,-1


In [ ]:
openClusters=openCleanupCluster.fit_predict(geoCleanup)
openCleanup['cluster']=openClusters
openCleanup['priority']=0

openClusters=openCollectionCluster.fit_predict(geoCollection)
openCollection['cluster']=openClusters
openCollection['priority']=0

openClusters=openDelRepRemCluster.fit_predict(geoDelRepRem)
openDelRepRem['cluster']=openClusters
openDelRepRem['priority']=0

openClusters=openSnowCluster.fit_predict(geoSnow)
openSnow['cluster']=openClusters
openSnow['priority']=0

openClusters=openVehTicCluster.fit_predict(geoVehTic)
openVehTic['cluster']=openClusters
openVehTic['priority']=0

In [ ]:
openCleanup.tail(3)

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,SERVICEDUEDATE,SERVICEORDERDATE,INSPECTIONFLAG,INSPECTIONDATE,INSPECTORNAME,SERVICEORDERSTATUS,STATUS_CODE,SERVICEREQUESTID,PRIORITY,STREETADDRESS,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS,DURATION,cluster,priority
84458,-77.024328,38.951747,118922,S0011,Alley Cleaning,SWMA- Solid Waste Management Admistration,DPW,1,2020-03-30 11:59:03+00:00,NaT,2020-05-13 11:59:03+00:00,2020-03-30 11:59:03+00:00,0.0,2020-04-03 06:37:03+00:00,NaN,Closed,NaN,20-00112964,Standard,5027 ILLINOIS AVENUE NW,397891.42,142649.06,38.951739,-77.024326,WASHINGTON,DC,20011.0,251802.0,4.0,NaN,0.405905,30,0
98643,-76.978819,38.917648,140371,GRAFF,Graffiti Removal,SWMA- Solid Waste Management Admistration,DPW,1,2020-03-07 21:13:24+00:00,NaT,2020-03-19 22:13:23+00:00,2020-03-07 21:13:24+00:00,0.0,2020-04-22 14:09:00+00:00,NaN,Closed,NaN,20-00083949,Standard,1720 NEW YORK AVENUE NE,401837.09,138863.74,38.917641,-76.978816,WASHINGTON,DC,20002.0,286427.0,5.0,NaN,3.182863,55,0
107124,-76.979343,38.919373,157348,GRAFF,Graffiti Removal,SWMA- Solid Waste Management Admistration,DPW,1,2020-03-07 21:14:55+00:00,NaT,2020-03-19 22:14:55+00:00,2020-03-07 21:14:55+00:00,0.0,2020-05-04 13:39:00+00:00,NaN,Closed,NaN,20-00083951,Standard,1725 W STREET NE,401791.56,139055.17,38.919365,-76.979341,WASHINGTON,DC,20018.0,56897.0,5.0,NaN,3.182819,55,0


In [ ]:
dispatchDPW= {}
dispatchDPW['dataframes']={'Cleanup': openCleanup, 'Collection': openCollection, 'DelRepRem': openDelRepRem, 'Snow': openSnow, 'VehTic': openVehTic}
dispatchDPW['OPTICS']={'Cleanup': openCleanupCluster, 'Collection': openCollectionCluster, 'DelRepRem': openDelRepRemCluster, 'Snow': openSnowCluster, 'VehTic': openVehTicCluster}
dispatchDPW['dataframes']['Cleanup'].head(2)

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,SERVICEDUEDATE,SERVICEORDERDATE,INSPECTIONFLAG,INSPECTIONDATE,INSPECTORNAME,SERVICEORDERSTATUS,STATUS_CODE,SERVICEREQUESTID,PRIORITY,STREETADDRESS,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS,DURATION,cluster,priority
2319,-76.996075,38.920478,3536,GRAFF,Graffiti Removal,SWMA- Solid Waste Management Admistration,DPW,1,2020-01-03 07:32:20+00:00,NaT,2020-01-17 07:32:20+00:00,2020-01-03 07:32:20+00:00,0.0,2020-08-31 11:42:00+00:00,NaN,Closed,NaN,20-00002831,Standard,919 RHODE ISLAND AVENUE NE,400340.60,139177.72,38.920471,-76.996072,WASHINGTON,DC,20018.0,294859.0,5.0,NaN,4.485101,0,0
3806,-77.023946,38.930157,5263,GRAFF,Graffiti Removal,SWMA- Solid Waste Management Admistration,DPW,1,2020-01-03 19:12:29+00:00,NaT,2020-01-17 19:12:29+00:00,2020-01-03 19:12:29+00:00,0.0,2020-04-13 08:19:00+00:00,NaN,Closed,NaN,20-00004487,Standard,707 KENYON STREET NW,397923.94,140252.42,38.930150,-77.023943,WASHINGTON,DC,20010.0,232953.0,1.0,NaN,4.479603,20,0


In [ ]:
closedDict={'Cleanup': closedCleanup.groupby(['cluster'])['DURATION'].mean(), 'Collection': closedCollection.groupby(['cluster'])['DURATION'].mean(), 'DelRepRem': closedDelRepRem.groupby(['cluster'])['DURATION'].mean(), 'Snow': closedSnow.groupby(['cluster'])['DURATION'].mean(), 'VehTic': closedVehTic.groupby(['cluster'])['DURATION'].mean()}
for key, value in closedDict.items():
  closedDict[key]=closedDict[key].to_frame()
  closedDict[key]=closedDict[key].reset_index()
  closedDict[key]['operation']=np.where(closedDict[key]['cluster']==-1,'iVal','avg')
  closedDict[key]=closedDict[key].rename(columns={'DURATION': 'operationValue'})
closedDict[key].head(2)

,cluster,operationValue,operation
0,-1,0.65747,iVal
1,0,0.00000,avg


In [ ]:
dispatchDPW['dataframes']['Cleanup'].head(2)

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,SERVICEDUEDATE,SERVICEORDERDATE,INSPECTIONFLAG,INSPECTIONDATE,INSPECTORNAME,SERVICEORDERSTATUS,STATUS_CODE,SERVICEREQUESTID,PRIORITY,STREETADDRESS,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS,DURATION,cluster,priority
2319,-76.996075,38.920478,3536,GRAFF,Graffiti Removal,SWMA- Solid Waste Management Admistration,DPW,1,2020-01-03 07:32:20+00:00,NaT,2020-01-17 07:32:20+00:00,2020-01-03 07:32:20+00:00,0.0,2020-08-31 11:42:00+00:00,NaN,Closed,NaN,20-00002831,Standard,919 RHODE ISLAND AVENUE NE,400340.60,139177.72,38.920471,-76.996072,WASHINGTON,DC,20018.0,294859.0,5.0,NaN,4.485101,0,0
3806,-77.023946,38.930157,5263,GRAFF,Graffiti Removal,SWMA- Solid Waste Management Admistration,DPW,1,2020-01-03 19:12:29+00:00,NaT,2020-01-17 19:12:29+00:00,2020-01-03 19:12:29+00:00,0.0,2020-04-13 08:19:00+00:00,NaN,Closed,NaN,20-00004487,Standard,707 KENYON STREET NW,397923.94,140252.42,38.930150,-77.023943,WASHINGTON,DC,20010.0,232953.0,1.0,NaN,4.479603,20,0


In [ ]:
# Prototype list of operations
# add means priority = duration + value
# multi means priority = duration * value
# avg means priority = (duration + value)/2
# iDur means priority = value
# iVal means priority = duration

for key, value in dispatchDPW['dataframes'].items():
  dispatchDPW['dataframes'][key]=dispatchDPW['dataframes'][key].merge(closedDict[key], on='cluster', how='left')
  dispatchDPW['dataframes'][key]['priority']=np.where(dispatchDPW['dataframes'][key]['operation']=="add", (dispatchDPW['dataframes'][key]['DURATION'] + dispatchDPW['dataframes'][key]['operationValue']), dispatchDPW['dataframes'][key]['priority'])
  dispatchDPW['dataframes'][key]['priority']=np.where(dispatchDPW['dataframes'][key]['operation']=="avg", (dispatchDPW['dataframes'][key]['DURATION'] + dispatchDPW['dataframes'][key]['operationValue'])/2, dispatchDPW['dataframes'][key]['priority'])
  dispatchDPW['dataframes'][key]['priority']=np.where(dispatchDPW['dataframes'][key]['operation']=="multi", (dispatchDPW['dataframes'][key]['DURATION'] * dispatchDPW['dataframes'][key]['operationValue']), dispatchDPW['dataframes'][key]['priority'])
  dispatchDPW['dataframes'][key]['priority']=np.where(dispatchDPW['dataframes'][key]['operation']=="iDur", (dispatchDPW['dataframes'][key]['operationValue']), dispatchDPW['dataframes'][key]['priority'])
  dispatchDPW['dataframes'][key]['priority']=np.where(dispatchDPW['dataframes'][key]['operation']=="iVal", (dispatchDPW['dataframes'][key]['DURATION']), dispatchDPW['dataframes'][key]['priority'])

In [ ]:
# We can sort the clusters and non-clusters by their mean priority into two groups, one meant for employees to handle in 
# quick succession such as spending the better part of the day working thru the requests in that neighborhood.
# The other isolated requests to be handled by the existing routing system for employees who have less time
# I'll just output the Cleanup data for this example

# First the isolated requests
dispatchCleanupIso=dispatchDPW['dataframes']['Cleanup'][dispatchDPW['dataframes']['Cleanup']['cluster']==-1]
dispatchCleanupIso=dispatchCleanupIso.sort_values('priority', ascending=False)
dispatchCleanupIso[['STREETADDRESS', 'SERVICECODE', 'SERVICECODEDESCRIPTION', 'priority', 'DETAILS', 'LATITUDE', 'LONGITUDE']].head(5) #This could be sent to dispatch

,STREETADDRESS,SERVICECODE,SERVICECODEDESCRIPTION,priority,DETAILS,LATITUDE,LONGITUDE
8,2065 PARK ROAD NW,GRAFF,Graffiti Removal,4.385819,NaN,38.935194,-77.045002
10,E STREET SW AND 2ND STREET SW,GRAFF,Graffiti Removal,4.359355,NaN,38.883262,-77.013695
13,500 - 799 BLOCK OF NEW JERSEY AVENUE SE,GRAFF,Graffiti Removal,4.345339,NaN,38.881703,-77.006045
22,801 2ND STREET SE,GRAFF,Graffiti Removal,4.310845,NaN,38.880333,-77.003775
23,M STREET NE AND 4TH STREET NE,NEIGHCLEANUP,Neighborhood Clean-Up,4.309680,NaN,38.905644,-77.000573


In [ ]:
# Now the requests around our OPTICS created neighborhoods
dispatchClusterCleanup=dispatchDPW['dataframes']['Cleanup'].groupby(['cluster'])['priority'].mean()
dispatchClusterCleanup=dispatchClusterCleanup.to_frame()
dispatchClusterCleanup=dispatchClusterCleanup.sort_values('priority', ascending=False)
dispatchClusterCleanup=dispatchClusterCleanup.reset_index()
dispatchDPW['dataframes']['Cleanup'][dispatchDPW['dataframes']['Cleanup']['cluster']==dispatchClusterCleanup['cluster'].iloc[0]][['STREETADDRESS', 'SERVICECODE', 'SERVICECODEDESCRIPTION', 'priority', 'DETAILS', 'LATITUDE', 'LONGITUDE']] #This could be sent to dispatch

,STREETADDRESS,SERVICECODE,SERVICECODEDESCRIPTION,priority,DETAILS,LATITUDE,LONGITUDE
3,K STREET NW AND NEW JERSEY AVENUE NW,S0391,Street Cleaning,4.284331,NaN,38.902514,-77.013722
14,NEW JERSEY AVENUE NW AND H STREET NW,GRAFF,Graffiti Removal,4.227653,NaN,38.900200,-77.012913
358,4TH STREET NW AND I STREET NW,S0391,Street Cleaning,2.219373,NaN,38.901325,-77.016174


In [ ]:
for key, value in dispatchDPW['dataframes'].items():
  print(str(key) + ": ")
  print(dispatchDPW['dataframes'][key]['SERVICECODE'].count())

Cleanup: 
430
Collection: 
1514
DelRepRem: 
370
Snow: 
118
VehTic: 
825
